In [4]:
# wunder parser
from bs4 import BeautifulSoup
from selenium import webdriver
import sys
import pandas as pd

#this is for testing only
station_id_list = ['CWSN', 'CYXU']
ride_day = 10
ride_month = 5
ride_year = 2004
station_index = 1

url = 'https://www.wunderground.com/history/daily/'+str(station_id_list[station_index])+ \
'/date/' + str(ride_year) + '-' + str(ride_month) + '-'+str(ride_day)


#url = 'https://www.wunderground.com/history/daily/CYXU/date/2004-5-10'

# instantiate webdriver and get data
# you need firefox bynary and the geckodriver installed
# the geckodriver interfaces with firefox to get the page and pass the data back
wd = webdriver.Firefox()
wd.get(url)

soup = BeautifulSoup(wd.page_source, 'lxml')

wd.quit()

In [5]:
#find all tables
data_found = False
tables = soup.find_all('table')
if not tables:
    data_found = False
else:
    # we get 3 tables: 0=empty; 1=records; 2=hourly data
    # we want only table 2

    table = tables[2]

    # parse table - first the header
    table_head = table.findAll('th')
    output_head = []
    for head in table_head:
        output_head.append(head.text.strip())

    # create empty dataframe
    weather_data = pd.DataFrame(columns=output_head)

    # parse rows
    output_rows = []
    rows = table.findAll('tr')
    for row in rows:
        columns = row.findAll('td')
        clean_row = []
        for column in columns:
            clean_row.append(column.text.strip().replace(u'\xa0', '') #this is to remove unwanted characters
                             .replace('mph', '') 
                             .replace('F', '')
                             .replace('in', ''))
        if len(clean_row) > 0: # discard row if it is empty
            output_rows.append(clean_row)

    # add it all to the dataframe
    weather_data = pd.DataFrame(columns=output_head, data=output_rows).apply(pd.to_numeric, errors='ignore')

    # substitute wind direction for numeric headings
    wind_dir_dict = {"N":0,"NNE":22.5,"NE":45,"ENE":67.5,
                     "E":90,"ESE":112.5, "SE":135, "SSE":157.5,
                     "S":180,"SSW":202.5,"SW":225,"WSW":247.5,
                     "W":270,"WNW":292.5,"NW":315,"NNW":337.5}

    weather_data['Wind_Dir'] = weather_data['Wind'].map(wind_dir_dict)

    #make time something useful
    weather_data['Time'] = pd.to_datetime(weather_data['Time'].apply(lambda x : str(ride_year)+
                                                                     '-'+str(ride_month)+
                                                                     '-'+str(ride_day)+
                                                                     ' '+x))
    data_found = True

In [6]:
data_found

True

In [ ]:
ss = []
if not ss:
    print(" it is empty")